In [58]:
from MIOFlow.models import make_model
import torch

In [59]:
batch_size = 4
feature_dim = 5
output_dim = 5

x = torch.randn(batch_size, feature_dim)
# m0 = torch.ones(batch_size, 1)
# xm = torch.cat([x, m0], dim=1)
t = torch.linspace(0, 1, 7)

model = make_model(feature_dims=feature_dim, which='ode_growth_rate')

In [116]:
t.shape

torch.Size([7])

In [60]:
xt, mt = model(x, t)

In [61]:
xt.shape, mt.shape

(torch.Size([4, 5]), torch.Size([4]))

In [62]:
xtseq, mtseq = model(x, t, return_whole_sequence=True)

In [63]:
xtseq.shape

torch.Size([7, 4, 5])

In [64]:
mtseq.shape

torch.Size([7, 4])

In [65]:
torch.nn.functional.softmax(mt, dim=-1) * mt.shape[-1]

tensor([0.9708, 1.0566, 1.2076, 0.7650], grad_fn=<MulBackward0>)

In [66]:
torch.exp(mt)/torch.exp(mt).sum(dim=0)

tensor([0.2427, 0.2641, 0.3019, 0.1913], grad_fn=<DivBackward0>)

In [67]:
torch.nn.functional.softmax(mtseq, dim=-1) * mtseq.shape[-1]

tensor([[1.0000, 1.0000, 1.0000, 1.0000],
        [0.9978, 1.0128, 1.0327, 0.9568],
        [0.9942, 1.0240, 1.0655, 0.9162],
        [0.9894, 1.0341, 1.0991, 0.8774],
        [0.9841, 1.0428, 1.1336, 0.8395],
        [0.9780, 1.0503, 1.1697, 0.8020],
        [0.9708, 1.0566, 1.2076, 0.7650]], grad_fn=<MulBackward0>)

In [68]:
xtseq.shape

torch.Size([7, 4, 5])

In [69]:
mtseq.unsqueeze(2).shape

torch.Size([7, 4, 1])

In [70]:
xmseq = torch.cat([xtseq, mtseq.unsqueeze(2)], dim=-1)
xmseq.shape

torch.Size([7, 4, 6])

In [71]:
dxdts = torch.stack([model.func(t[i], xmseq[i]) for i in range(len(t))])

In [72]:
dxdts.shape

torch.Size([7, 4, 6])

In [73]:
mt.unsqueeze(1).shape

torch.Size([4, 1])

In [74]:
model.func

ToyODE(
  (seq): Sequential(
    (0): Linear(in_features=9, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=6, bias=True)
  )
)

In [75]:
xt.shape

torch.Size([4, 5])

In [76]:
mt.shape

torch.Size([4])

In [77]:
xm = torch.cat([xt, mt.unsqueeze(-1)], dim=-1)
dxdt = model.func(t[-1], xm)

In [78]:
dxdt[...,:-1].shape

torch.Size([4, 5])

In [79]:
mtseq.shape

torch.Size([7, 4])

In [80]:
xtseq.shape

torch.Size([7, 4, 5])

In [81]:
t.shape

torch.Size([7])

In [82]:
t[[0,1,2],...]

tensor([0.0000, 0.1667, 0.3333])

In [83]:
mt.shape

torch.Size([4])

In [84]:
(dxdt * mt.unsqueeze(-1)).shape

torch.Size([4, 6])

In [85]:
dxdts.shape

torch.Size([7, 4, 6])

In [86]:
mtseq.shape

torch.Size([7, 4])

In [87]:
(dxdts * mtseq.unsqueeze(-1)).shape

torch.Size([7, 4, 6])

In [88]:
torch.nn.functional.softmax(mtseq, dim=-1).shape

torch.Size([7, 4])

In [89]:
mtseq.shape[-1]

4

In [90]:
torch.nn.functional.softmax(mt, dim=-1).shape

torch.Size([4])

In [91]:
mt.shape[-1]

4

In [92]:
def func_end_0(t, x):
    xm = model.func(t, x)
# mt[[0,2,4,6,8]] = 0.
model.func(t[-1], xm).shape

torch.Size([4, 6])

In [93]:
xm[...,-1].shape

torch.Size([4])

In [94]:
xm.shape

torch.Size([4, 6])

In [95]:
xmseq.shape

torch.Size([7, 4, 6])

In [96]:
xm[[0,2],-1] = 0.

In [97]:
xm

tensor([[ 0.7460, -0.5456, -1.0027, -0.4393, -0.3585,  0.0000],
        [-1.2139, -0.1798,  0.1550,  1.0991, -0.5845,  1.1864],
        [ 0.6480, -1.6592, -3.3590, -0.8466, -0.2299,  0.0000],
        [-0.1148,  0.4754, -0.4451, -1.1064,  1.1586,  0.8635]],
       grad_fn=<CopySlices>)

In [98]:
dxmdt = model.func(t[-1], xm)

In [99]:
dxmdt

tensor([[ 0.1114, -0.4193, -0.2571, -0.2891, -0.1134,  0.1890],
        [ 0.1237, -0.1615,  0.0009, -0.1757, -0.2481,  0.1611],
        [ 0.0801, -0.6372, -0.8091, -0.5411, -0.1027,  0.3390],
        [ 0.1650, -0.2092, -0.0134, -0.3447, -0.4835, -0.0278]],
       grad_fn=<AddmmBackward0>)

In [100]:
xm.shape

torch.Size([4, 6])

In [101]:
xt.shape

torch.Size([4, 5])

In [102]:
xtseq.shape

torch.Size([7, 4, 5])

In [103]:
xm.shape

torch.Size([4, 6])

In [104]:
xmseq.shape

torch.Size([7, 4, 6])

In [105]:
xmseq.shape

torch.Size([7, 4, 6])

In [106]:
xm.shape

torch.Size([4, 6])

In [107]:
xtseq.shape

torch.Size([7, 4, 5])

In [108]:
xtseq.norm(dim=-1).shape

torch.Size([7, 4])

In [109]:
(xtseq / xtseq.norm(dim=-1, keepdim=True)).shape

torch.Size([7, 4, 5])

In [110]:
torch.diff(xtseq / xtseq.norm(dim=-1, keepdim=True), dim=0).shape

torch.Size([6, 4, 5])

In [111]:
torch.diff(xtseq / xtseq.norm(dim=-1, keepdim=True), dim=0)

tensor([[[-2.3809e-02, -3.1514e-02, -8.2331e-03, -1.3875e-02,  2.8666e-03],
         [-1.9941e-03, -1.3576e-02, -5.4659e-03, -7.7661e-03, -2.4443e-02],
         [-9.2892e-03, -5.2105e-03,  2.5960e-03, -1.4812e-02, -9.9999e-04],
         [ 1.0675e-02, -3.6855e-03, -3.9056e-04, -3.3850e-02, -1.4750e-02]],

        [[-2.2008e-02, -2.9510e-02, -3.9362e-03, -1.5483e-02,  1.7417e-03],
         [-6.5929e-04, -1.5102e-02, -4.3810e-03, -8.3821e-03, -2.6042e-02],
         [-8.4651e-03, -5.5358e-03,  3.2527e-03, -1.4406e-02, -1.3003e-03],
         [ 1.2104e-02, -5.5695e-03, -2.6271e-04, -3.6062e-02, -1.6988e-02]],

        [[-2.0293e-02, -2.8098e-02, -2.8104e-04, -1.6850e-02,  1.5502e-03],
         [ 6.8498e-04, -1.6469e-02, -3.1326e-03, -9.1614e-03, -2.7284e-02],
         [-7.6881e-03, -5.8275e-03,  3.8420e-03, -1.3961e-02, -1.5218e-03],
         [ 1.3884e-02, -7.6974e-03, -4.4167e-05, -3.8176e-02, -1.9553e-02]],

        [[-1.9098e-02, -2.7257e-02,  2.6186e-03, -1.7429e-02,  1.5830e-03],
      

In [112]:
torch.square(torch.diff(xtseq / xtseq.norm(dim=-1, keepdim=True), dim=0)).mean()

tensor(0.0003, grad_fn=<MeanBackward0>)

In [113]:
torch.diff(xtseq / xtseq.norm(dim=-1, keepdim=True), dim=0)

tensor([[[-2.3809e-02, -3.1514e-02, -8.2331e-03, -1.3875e-02,  2.8666e-03],
         [-1.9941e-03, -1.3576e-02, -5.4659e-03, -7.7661e-03, -2.4443e-02],
         [-9.2892e-03, -5.2105e-03,  2.5960e-03, -1.4812e-02, -9.9999e-04],
         [ 1.0675e-02, -3.6855e-03, -3.9056e-04, -3.3850e-02, -1.4750e-02]],

        [[-2.2008e-02, -2.9510e-02, -3.9362e-03, -1.5483e-02,  1.7417e-03],
         [-6.5929e-04, -1.5102e-02, -4.3810e-03, -8.3821e-03, -2.6042e-02],
         [-8.4651e-03, -5.5358e-03,  3.2527e-03, -1.4406e-02, -1.3003e-03],
         [ 1.2104e-02, -5.5695e-03, -2.6271e-04, -3.6062e-02, -1.6988e-02]],

        [[-2.0293e-02, -2.8098e-02, -2.8104e-04, -1.6850e-02,  1.5502e-03],
         [ 6.8498e-04, -1.6469e-02, -3.1326e-03, -9.1614e-03, -2.7284e-02],
         [-7.6881e-03, -5.8275e-03,  3.8420e-03, -1.3961e-02, -1.5218e-03],
         [ 1.3884e-02, -7.6974e-03, -4.4167e-05, -3.8176e-02, -1.9553e-02]],

        [[-1.9098e-02, -2.7257e-02,  2.6186e-03, -1.7429e-02,  1.5830e-03],
      

In [114]:
dxdts.norm(dim=-1) - torch.arange(7).unsqueeze(-1)

tensor([[ 0.4249,  0.3007,  0.9810,  0.6314],
        [-0.5696, -0.6919,  0.0062, -0.3660],
        [-1.5598, -1.6783, -0.9677, -1.3664],
        [-2.5416, -2.6617, -1.9404, -2.3664],
        [-3.5198, -3.6440, -2.9133, -3.3636],
        [-4.4969, -4.6234, -3.8846, -4.3573],
        [-5.4707, -5.6003, -4.8533, -5.3485]], grad_fn=<SubBackward0>)

In [117]:
dxdts.shape

torch.Size([7, 4, 6])

In [124]:
%timeit dxdts.norm(dim=-1)

9.67 µs ± 579 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [125]:
%timeit torch.sqrt(torch.square(dxdts).sum(dim=-1))

15.4 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
